In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain.chat_models import ChatOpenAI


In [ ]:
## Environmental value from .env file
from dotenv import load_dotenv
load_dotenv()

In [ ]:
KEY= os.getenv("API_KEY")

In [ ]:
print(KEY)

### Initiate LLM

In [ ]:
llm= ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=1.2)

In [ ]:
llm

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


### Design Input and Output Prompts

In [ ]:
TEMPLATE="""
Text: {Text}
You are an expert information provider. Given above text, it is your job to provide appropritae answer in {tone} for the questions \
    asked. Make sure your \ response is in text utf-8 format.
    Question: {question}
"""

In [ ]:
policy_read_prompt= PromptTemplate(
    input_variables= ['text', 'tone', 'question'],
    template= TEMPLATE
)

In [ ]:
policy_read_chain= LLMChain(llm= llm, prompt=policy_read_prompt, output_key="policy_response", verbose=True)

Create Sequential chain

In [ ]:
generate_evaluate_chain= SequentialChain(
    chains= [policy_read_chain],
    input_variables= ['Text', 'tone', 'question'],
    output_variables= ["policy_response"],
    verbose= True
)

#### Combine Leave and travel policy

In [ ]:
##with open('/Users/vikaslakka/Desktop/FSDS/GenAI/mcq_generator/data.txt', 'r') as file:
    ##TEXT= file.read()
## Leave
with open('/Users/vikaslakka/Desktop/FSDS/GenAI/Hackathon/Policy_bot/Leave_Policy.txt', 'r') as file:
    LEAVE_TEXT= file.read()    

## Travel
with open('/Users/vikaslakka/Desktop/FSDS/GenAI/Hackathon/Policy_bot/Travel Policy.txt', 'r') as file:
    TRAVEL_TEXT= file.read()   
##Combine both
TEXT= LEAVE_TEXT + "\n\n" + TRAVEL_TEXT

In [ ]:
print(TEXT)

initiating Chain

In [ ]:
TONE="medium"
QUESTION="How do i have to submit travel request in advance?"

In [ ]:
with get_openai_callback() as cb:
    response= generate_evaluate_chain(
        {
            "Text":TEXT,
            "tone":TONE,
            "question": QUESTION
        }
    )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
policy_answer= response.get("policy_response")

In [ ]:
policy_answer

In [ ]:
response